### Load R and install necessary packages


In [ ]:
%load_ext rpy2.ipython

In [13]:
%%R

#install.packages("tibble")
#install.packages("dplyr")


library(tibble)
library(dplyr)


### List desired gmIDs


In [ ]:
%%R

gmIDs <- c(
    "feaf2ba8-d28d-11ee-b437-336917683bb8"
           )
print(gmIDs)


### Define useful functions


In [15]:
%%R

binary_driving_mode <- function (df_chassis) {
    result <- ifelse(df_chassis$drivingMode == "COMPLETE_AUTO_DRIVE", 1, 0)

    return(result)
}

merge_chassis_pose <- function (df_chassis, df_pose) {
    
    
    df_chassis <- df_chassis[order(df_chassis$time), ]
    df_pose <- df_pose[order(df_pose$time), ]
    
    #make sure data is sorted by time fist

    chassis_time_arr <- df_chassis$time
    pose_time_arr <- df_pose$time

    matched_vector <- c()

    for (chassis_time in chassis_time_arr) {
        abs_time_diff_arr <- abs(pose_time_arr - chassis_time)

        min_index <-  which(abs_time_diff_arr == min(abs_time_diff_arr))
        
        matched_vector <-  c(matched_vector, pose_time_arr[min_index])
    }



    #df_chassis$matched <- df_pose$time[which(abs(pose_time_arr - df_chassis$time) == min(abs(pose_time_arr - df_chassis$time)))]
    df_chassis$matched <-  matched_vector
    df_pose$matched <-  df_pose$time


    return (list(df_chassis = df_chassis, df_pose = df_pose))


}

### Read chassis and best_pose data into one large tibble


In [ ]:
%%R

df_merged <- tibble()

for (id in gmIDs) {
    
    print(id)
    #temp_chassis <- read_csv(sprintf("./data/chassis/%s.csv", id), show_col_types = FALSE)
    #temp_pose <- read_csv(sprintf("./data/best_pose/%s.csv", id),show_col_types = FALSE)
    #temp_chassis <- as_tibble(read.csv(sprintf("./data/Raw_Data/%s/_apollo_canbus_chassis/%s_apollo_canbus_chassis.csv", id, id)))
    #temp_pose <- as_tibble(read.csv(sprintf("./data/Raw_Data/%s/_apollo_sensor_gnss_best_pose/%s_apollo_sensor_gnss_best_pose.csv", id, id)))
    
    temp_chassis <- as_tibble(read.csv(sprintf("../data/%s/_apollo_canbus_chassis/%s_apollo_canbus_chassis.csv", id, id)))
    temp_pose <- as_tibble(read.csv(sprintf("../data/%s/_apollo_sensor_gnss_best_pose/%s_apollo_sensor_gnss_best_pose.csv", id, id)))
    
    
    results <- merge_chassis_pose(temp_chassis,temp_pose)

    result_full <- full_join(results$df_chassis,results$df_pose, by = "matched")


    result_full$binaryDrivingMode <- binary_driving_mode(result_full)

    df_merged <- bind_rows(df_merged, result_full)
}

print("done loading data")


In [ ]:
%%R

glimpse(df_merged)

### Check distribution of driving modes. 0 = manual, 1 = automatic


In [ ]:
%%R
table(df_merged$binaryDrivingMode)